<a href="https://colab.research.google.com/github/Natalia409634/Grupo7-practica2-ML/blob/main/G7_P2_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRÁCTICA 2 -- APRENDIZAJE AUTOMÁTICO

# Librerías

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pickle
import os

# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


# Lectura de Datos

Los datos están en formato 'pkl'

In [2]:
# Carga el archivo pkl en un DataFrame de Pandas
data_df = pd.read_pickle('/content/drive/MyDrive/Aprendizaje Automático/practica 2/datos_grupos/attrition_available_7.pkl')

Edición visual de los datos

In [3]:
def set_frame_style(df, caption=""):
    """Helper function to set dataframe presentation style.
    """
    return df.style.background_gradient(cmap='Blues').set_caption(caption).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'darkblue'),
        ('font-size', '18px'),
        ('font-weight','bold')
    ]}])

# EDA Simplificado

Hacer un EDA muy simplificado: 

*   cuántas instancias / cuantos
atributos y de qué tipo (numéricos, ordinales, categóricos) DONE
*   columnas constantes o
innecesarias DONE
*   que proporción de missing values por atributo
*   tipo de problema:
(clasificación o regresión) -> clasificación no?
*   ¿es desbalanceado?









In [4]:
data_df.head()

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
1,7.315971,13.0,2.0,NaN,3.0,2.0,NaN,31.0,Yes,Travel_Frequently,...,0.0,Y,NaN,NaN,1,6.0,3.0,5.0,1.0,4.0
6,6.450877,17.0,3.0,4.0,1.0,3.0,1.0,28.0,Yes,Travel_Rarely,...,2.0,Y,20.0,NaN,1,5.0,2.0,0.0,0.0,0.0
13,8.871421,14.0,2.0,3.0,1.0,2.0,2.0,47.0,Yes,NaN,...,1.0,Y,NaN,8.0,2,10.0,4.0,10.0,9.0,9.0
28,10.713066,6.0,2.0,3.0,4.0,3.0,1.0,44.0,Yes,Travel_Frequently,...,3.0,Y,14.0,8.0,1,19.0,2.0,1.0,0.0,0.0
30,9.662808,11.0,2.0,NaN,1.0,2.0,NaN,26.0,Yes,Travel_Rarely,...,2.0,Y,NaN,8.0,0,5.0,5.0,3.0,0.0,2.0


In [5]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 1 to 4409
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hrs                      3588 non-null   float64
 1   absences                 3663 non-null   float64
 2   JobInvolvement           3542 non-null   float64
 3   PerformanceRating        3684 non-null   float64
 4   EnvironmentSatisfaction  4385 non-null   float64
 5   JobSatisfaction          3484 non-null   float64
 6   WorkLifeBalance          3526 non-null   float64
 7   Age                      3678 non-null   float64
 8   Attrition                4410 non-null   object 
 9   BusinessTravel           3452 non-null   object 
 10  Department               3553 non-null   object 
 11  DistanceFromHome         3569 non-null   float64
 12  Education                3676 non-null   float64
 13  EducationField           3441 non-null   object 
 14  EmployeeCount           

EL conjunto de datos proporcionado tiene 30 atributos, los cuales adoptan un formato ':float64', 'int64' y 'object'.

Identificar y tratar datos faltantes

In [21]:
missing_values_count = data_df.isnull().sum().sort_values(ascending=False)
prop_miss = data_df.isna().mean().sort_values(ascending=False)

missing_props_df = pd.concat([missing_values_count, prop_miss], axis=1, keys=['Missing Values Count', 'Proportion Missing'])

print(missing_props_df)


                         Missing Values Count  Proportion Missing
EducationField                            969            0.219728
MonthlyIncome                             962            0.218141
BusinessTravel                            958            0.217234
PercentSalaryHike                         953            0.216100
StandardHours                             945            0.214286
JobSatisfaction                           926            0.209977
TotalWorkingYears                         896            0.203175
WorkLifeBalance                           884            0.200454
JobInvolvement                            868            0.196825
Department                                857            0.194331
EmployeeCount                             849            0.192517
JobLevel                                  844            0.191383
DistanceFromHome                          841            0.190703
hrs                                       822            0.186395
YearsAtCom

Aquí se puede encontrar el numero de datos faltantes ordenadors de mayor a menos por atributo. Las columnas más incompletas son EducationField, MonthlyIncome Y BusinessTravel. **menos datos = menos precisión**

Asimismo, tambien se muestra la proporción de valores faltantes en cada columna del DataFrame.

Por ejemplo, para la columna "hrs" la proporción de valores faltantes es del 18.64%, lo que significa que aproximadamente el 18.64% de los valores en esa columna están faltando. De manera similar, para la columna "absences", la proporción de valores faltantes es del 16.94%, lo que significa que alrededor del 16.94% de los valores en esa columna están faltando.

También se puede observar como los atributos 'Attrition','StockOptionLevel', 'EmployeeID' y 'Over18'tienen un valor de 0.00, lo que implica que estan presentes todos los valores 

Guardaremos el dataframe en un nuevo conjunto de datos (newData-df). PREGUNTAR AL PROFE SI DEBEMOS QUEDARNOS EL NUEVO O PERDERIAMOS MUCHA INFO

In [27]:
newData_df= data_df.dropna()
print(newData_df.info)

<bound method DataFrame.info of             hrs  absences  JobInvolvement  PerformanceRating  \
28    10.713066       6.0             2.0                3.0   
2013   9.875726      15.0             3.0                3.0   
3067   7.686566      15.0             2.0                3.0   
3296   7.444562      18.0             3.0                4.0   
3461   8.037368      19.0             3.0                4.0   
280    5.771757      16.0             3.0                3.0   
514    6.468416      16.0             3.0                3.0   
1230   7.629565      12.0             3.0                3.0   
1511   6.994158      19.0             2.0                3.0   
1518   6.428416      20.0             3.0                4.0   
1608   8.253846      19.0             3.0                3.0   
1657   7.160385       7.0             3.0                3.0   
1755   5.852605      11.0             2.0                3.0   
1924   6.972995      18.0             3.0                3.0   
3216   6

In [28]:
cols = data_df.columns.to_list()

display(set_frame_style(data_df[cols].head(),'Primeros'))

display(set_frame_style(data_df[cols].describe(),'Resumen de estadistica'))

display(set_frame_style(data_df[cols].isna().any().to_frame().transpose(), 'Columnas con valor nulo'))

display(set_frame_style(data_df[cols].nunique().to_frame().rename({0:'Numero de valores únicos'}, axis=1).transpose(), 'Valores únicos en cada variable'))

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
1,7.315971,13.000000,2.000000,nan,3.000000,2.000000,nan,31.000000,Yes,Travel_Frequently,Research & Development,10.000000,1.000000,Life Sciences,1.000000,2,Female,nan,Research Scientist,Single,41890.000000,0.000000,Y,nan,nan,1,6.000000,3.000000,5.000000,1.000000,4.000000
6,6.450877,17.000000,3.000000,4.000000,1.000000,3.000000,1.000000,28.000000,Yes,Travel_Rarely,nan,11.000000,2.000000,Medical,1.000000,7,Male,2.000000,nan,Single,58130.000000,2.000000,Y,20.000000,nan,1,5.000000,2.000000,0.000000,0.000000,0.000000
13,8.871421,14.000000,2.000000,3.000000,1.000000,2.000000,2.000000,47.000000,Yes,nan,Research & Development,1.000000,1.000000,Medical,1.000000,14,Male,1.000000,Research Scientist,Married,57620.000000,1.000000,Y,nan,8.000000,2,10.000000,4.000000,10.000000,9.000000,9.000000
28,10.713066,6.000000,2.000000,3.000000,4.000000,3.000000,1.000000,44.000000,Yes,Travel_Frequently,Research & Development,1.000000,2.000000,Medical,1.000000,29,Male,2.000000,Research Scientist,Divorced,103330.000000,3.000000,Y,14.000000,8.000000,1,19.000000,2.000000,1.000000,0.000000,0.000000
30,9.662808,11.000000,2.000000,nan,1.000000,2.000000,nan,26.000000,Yes,Travel_Rarely,Research & Development,4.000000,nan,Medical,1.000000,31,Male,3.000000,Research Scientist,nan,nan,2.000000,Y,nan,8.000000,0,5.000000,5.000000,3.000000,0.000000,2.000000


,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,DistanceFromHome,Education,EmployeeCount,EmployeeID,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
count,3588.000000,3663.000000,3542.000000,3684.000000,4385.000000,3484.000000,3526.000000,3678.000000,3569.000000,3676.000000,3561.000000,4410.000000,3566.000000,3448.000000,3646.000000,3457.000000,3465.000000,4410.000000,3514.000000,3700.000000,3628.000000,3652.000000,3665.000000
mean,7.325269,12.731095,2.723038,3.155266,2.723603,2.733639,2.758934,36.942360,9.177641,2.909412,1.000000,2205.500000,2.046831,65417.700116,2.715030,15.244721,8.000000,0.793878,11.268355,2.795676,6.995039,2.204272,4.116235
std,1.332110,5.531700,0.718360,0.362207,1.092756,1.095605,0.701183,9.124196,8.093058,1.023644,0.000000,1273.201673,1.094571,47530.589620,2.511696,3.682925,0.000000,0.851883,7.785224,1.290212,6.084324,3.244261,3.557025
min,5.416880,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,18.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10090.000000,0.000000,11.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.295553,8.000000,2.000000,3.000000,2.000000,2.000000,2.000000,30.000000,2.000000,2.000000,1.000000,1103.250000,1.000000,29290.000000,1.000000,12.000000,8.000000,0.000000,6.000000,2.000000,3.000000,0.000000,2.000000
50%,7.018052,13.000000,3.000000,3.000000,3.000000,3.000000,3.000000,36.000000,7.000000,3.000000,1.000000,2205.500000,2.000000,49330.000000,2.000000,14.000000,8.000000,1.000000,10.000000,3.000000,5.000000,1.000000,3.000000
75%,7.912442,17.000000,3.000000,3.000000,4.000000,4.000000,3.000000,43.000000,14.000000,4.000000,1.000000,3307.750000,3.000000,84120.000000,4.000000,18.000000,8.000000,1.000000,15.000000,3.000000,9.000000,3.000000,7.000000
max,10.937261,24.000000,4.000000,4.000000,4.000000,4.000000,4.000000,60.000000,29.000000,5.000000,1.000000,4410.000000,5.000000,199990.000000,9.000000,25.000000,8.000000,3.000000,40.000000,6.000000,40.000000,15.000000,17.000000


,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,False,True,True,True,True,True,True,False,True,True,False,True,True,True,True,True


,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
Numero de valores únicos,3585,24,4,2,4,4,4,43,2,3,3,29,5,6,1,4410,2,5,9,3,1337,10,1,15,1,4,40,7,37,16,18


¿Problema de clasificación o de regresión?

Se trat de un **problema de clasificación**, ya que la variable de salida que se espera del modelo es categórica, es decir, se espera que el modelo prediga si un empleado abandonará o no la empresa en un futuro cercano

**¿El problema está balanceado?**

In [30]:
counts = data_df.apply(pd.value_counts).fillna(0)
print(counts)

                   hrs  absences  JobInvolvement  PerformanceRating  \
0.0                0.0       0.0             0.0                0.0   
1.0                0.0       8.0           213.0                0.0   
2.0                0.0      31.0           901.0                0.0   
3.0                0.0      61.0          2082.0             3112.0   
4.0                0.0     125.0           346.0              572.0   
...                ...       ...             ...                ...   
Technical Degree   0.0       0.0             0.0                0.0   
Travel_Frequently  0.0       0.0             0.0                0.0   
Travel_Rarely      0.0       0.0             0.0                0.0   
Y                  0.0       0.0             0.0                0.0   
Yes                0.0       0.0             0.0                0.0   

                   EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance  \
0.0                                    0.0              0.0        

EJ-> La fila con el índice "1.0" representa los valores medios de todas las variables para los empleados que tienen 8 ausencias durante el año. También se puede ver que los empleados en la fila "1.0" tienen un nivel de satisfacción laboral de 670 y un nivel de satisfacción ambiental de 845, entre otros valores.

Es posible comparar el nivel de satisfacción laboral de los empleados que faltaron mucho con los que faltaron poco. También es posible identificar cualquier relación entre las variables para cada categoría.

EN general, parece balanceado-> PREGUNTAR

## 7. Separación en conjuntos

Seguirá la siguiente distribución:

*   Cojunto de entrenamiento [X_train, y_train]
*   Conjunto de validación [X_valid, y_valid]
*   Conjunto de test [X_test, y_test]

In [12]:
from sklearn.model_selection import train_test_split

X = data_df.drop(columns = [-1])
y = data_df[-1]

X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.67, random_state=7)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=7)

KeyError: ignored

Realizamos la separación en conjuntos empleando train_test_split siguiendo las recomendaciones del profesor dadas en clase.

# Evaluación

Simplemente
dividiremos los datos en un conjunto de train para entrenar y ajustar hiper-Aprendizaje Automático
Práctica 2: Predicción de burnout
parámetros, y un conjunto de test en el que evaluaremos las distintas posibilidades
que se probarán en la práctica. Hay que recordar que En problemas de clasificación
desbalanceados hay que usar particiones estratificadas y métricas adecuadas
(balanced_accuracy, f1, matriz de confusión). También es conveniente que los
métodos de construcción de modelos traten el desbalanceo, usando por ejemplo
el parámetro class_weight=”balanced”.

# Construcción de modelos

Construcción de modelos: para esta práctica usaremos
LogisticRegression como método base (sin ajustar hiper-parámetros) y Boosting
como método avanzado (ajustando hiper-parámetros), a elegir. Es importante
realizar los preprocesos que los datos necesiten, usando preferentemente
pipelines. Como método de boosting, se puede elegir uno de entre los métodos de
boosting disponibles en scikit-learn. Si además se usa uno de entre las librerías
externas xgboost, lightgbm o catboost, se pueden sacar +0.35 puntos adicionales.

## Logistic Regression

Los parámetros más importantes de la implantación de sklearn (`LogisticRegression`) son:

- `penalty`: El tipo de aplicación de regularización.
- `C`: (por defecto 1.0) Inverso de la fuerza de regularización; Valores más pequeños especifican una regularización más fuerte.
- `solver`: Algoritmo a utilizar en el problema de optimización.

In [ ]:
# Probamos el modelo sin aplicar ninguna regularización y con los parámetros por efecto
# ==============================================================================
scaler = StandardScaler()
lr = LogisticRegression(penalty='none', random_state = 7)

pipe_scale_lr = Pipeline([
    ('scale', scaler),
    ('lr', lr)])

# Entrenamiento del modelo
# ==============================================================================
cv = KFold(n_splits=5, shuffle=True, random_state = 42 )
scores = cross_val_score(pipe_scale_lr, X_train, y_train, scoring='accuracy', cv = cv)
print(f"All the accuracies are: {scores}")
print(f"And the average crossvalidation accuracy is: {scores.mean():.2f} +- {scores.std():.2f}")

In [ ]:
param_grid = [{'lr__penalty': ['l1', 'l2', 'elascticnet'],
               'lr__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
               'lr__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}]


inner = KFold(n_splits=3, shuffle=True, random_state=42)

#budget = 40
# Cross-validation (3-fold) para la búsqueda de hiper-parámetros
clf = GridSearchCV (estimator  = pipe_scale_lr,
                    param_grid = param_grid,
                    scoring='accuracy',
                    cv=inner,
                    refit=True,
                    n_jobs=-1,
                    verbose=1,
                    return_train_score=True)

np.random.seed(42)

clf.fit(X=X_train, y=y_train)

In [ ]:
resultados = pd.DataFrame(clf.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head()

In [ ]:
clf.best_params_, clf.best_score_

In [ ]:
# Información del modelo
# ==============================================================================
modelo_final = clf.best_estimator_
print("Intercept:", modelo_final['lr'].intercept_)
print("Coeficientes:", list(zip(df.columns, modelo_final['lr'].coef_.flatten(), )))
print("Accuracy de test:", modelo_final.score(X_test, y_test))

In [ ]:
y_test_pred = modelo_final.predict(X_test)
result = metrics.classification_report(y_test, y_test_pred)
print("Classification Report:",)
print (result)

In [ ]:
# Creates a confusion matrix
cm = metrics.confusion_matrix(y_test, y_test_pred)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
# Transform to df for easier plotting
cm_df = pd.DataFrame(cm,
                     index = ['Benigno','Maligno'],
                     columns = ['Benigno','Maligno'])
plt.figure(figsize=(5.5,4))
sns.heatmap(cm_df, annot=True)
plt.title('Accuracy:{0:.3f}'.format(accuracy))
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
#Obtenemos las curva ROC y el área bajo la curva (AUC)

probs = modelo_final.predict_proba(X_test)[:, 1]

auc = metrics.roc_auc_score(y_test, probs)
fpr, tpr, thresholds = metrics.roc_curve(y_test, probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr, tpr, label=f'AUC  = {auc:.2f}')
plt.plot([0, 1], [0, 1], color='blue', linestyle='--', label='Baseline')
plt.title('Curva ROC', size=20)
plt.xlabel('Falsos Positivos', size=14)
plt.ylabel('Verdaderos Positivos', size=14)
plt.legend();

# Mejora de resultados

Usando algún método de selección de atributos de tipo filter
(SelectKBest) de entre los disponibles en sklearn (f_classif,
mutual_info_classif o chi2), comprobad si se pueden mejorar los resultados del
apartado anterior y extraer conclusiones sobre qué atributos son más importantes,
al menos de acuerdo a estos métodos.